In [ ]:

!pip install arch
from arch import arch_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 26.8 MB/s eta 0:00:00


In [96]:
import tensorflow as tf
from tensorflow.keras import layers, models,Input
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
def load_data(symbol="SPY", start="1994-01-01", end="2021-01-01", interval='1d'):
    df = yf.download(symbol, start=start, end=end, interval=interval)
    df.reset_index(inplace=True)
    return df

def preprocess_data(df):
    df = df[['Date', 'Close']]
    df.dropna(inplace=True)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

def train_test(list_df, train_size=0.7, test_size=0.3):
    result = []
    for df in list_df:
        X = df.index.values.reshape(-1, 1)
        y = df['Close'].values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, train_size=train_size, test_size=test_size, shuffle=False
        )
        result.append([X_train, X_test, y_train, y_test])
    return result

# def etl_pipeline(train_size=0.7, test_size=0.3):
#     gspc = load_data('^GSPC')
#     dji = load_data('^DJI')
#     nyse = load_data('^NYA')

#     gspc_close = preprocess_data(gspc)
#     dji_close = preprocess_data(dji)
#     nyse_close = preprocess_data(nyse)

#     list_df_close = [gspc_close, dji_close, nyse_close]

#     splits = train_test(list_df_close, train_size=train_size, test_size=test_size)

#     return {
#         "GSPC": {"X_train": splits[0][0], "X_test": splits[0][1], "y_train": splits[0][2], "y_test": splits[0][3]},
#         "DJI":  {"X_train": splits[1][0], "X_test": splits[1][1], "y_train": splits[1][2], "y_test": splits[1][3]},
#         "NYSE": {"X_train": splits[2][0], "X_test": splits[2][1], "y_train": splits[2][2], "y_test": splits[2][3]},
#     }

import pandas as pd

def etl_pipeline(train_size=0.7, test_size=0.3):
    # Load data
    gspc = load_data('^GSPC')
    dji = load_data('^DJI')
    nyse = load_data('^NYA')

    # Preprocess data
    gspc_close = preprocess_data(gspc)
    dji_close = preprocess_data(dji)
    nyse_close = preprocess_data(nyse)

    list_df_close = [gspc_close, dji_close, nyse_close]

    # Function to split data and shift by 90 days
    def train_test_shift(df, train_size, shift_days=90):
        # Split data into train and test without shuffling
        train_size = int(len(df) * train_size)
        X_train = df.iloc[:train_size]
        X_test = df.iloc[train_size:]

        # Shift the data by 90 days to create y
        y_train = df.iloc[shift_days:train_size + shift_days]
        y_test = df.iloc[train_size + shift_days:]

        # Ensure X and y have the same length
        X_train = X_train.iloc[:-shift_days]
        X_test = X_test.iloc[:-shift_days]

        return X_train, X_test, y_train, y_test

    # Apply train_test_shift to each dataframe
    splits = [train_test_shift(df, train_size) for df in list_df_close]

    return {
        "GSPC": {"X_train": splits[0][0], "X_test": splits[0][1], "y_train": splits[0][2], "y_test": splits[0][3]},
        "DJI":  {"X_train": splits[1][0], "X_test": splits[1][1], "y_train": splits[1][2], "y_test": splits[1][3]},
        "NYSE": {"X_train": splits[2][0], "X_test": splits[2][1], "y_train": splits[2][2], "y_test": splits[2][3]},
    }


In [ ]:
gspc=load_data('^GSPC')
dji=load_data('^DJI')
nyse=load_data('^NYA')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
gspc_close=preprocess_data(gspc)
dji_close=preprocess_data(dji)
nyse_close=preprocess_data(nyse)

<ipython-input-5-6aa10446cf8a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-5-6aa10446cf8a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
<ipython-input-5-6aa10446cf8a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-5-6aa10446cf8a>:9: SettingWithCopyWarni

In [ ]:
print(gspc_close)

Price             Close
Ticker            ^GSPC
Date                   
1994-01-03   465.440002
1994-01-04   466.890015
1994-01-05   467.549988
1994-01-06   467.119995
1994-01-07   469.899994
...                 ...
2020-12-24  3703.060059
2020-12-28  3735.360107
2020-12-29  3727.040039
2020-12-30  3732.040039
2020-12-31  3756.070068

[6799 rows x 1 columns]


In [ ]:
# Run the ETL pipeline
result = etl_pipeline()

# Assign GSPC values
X_gspc_train = result["GSPC"]["X_train"]
X_gspc_test  = result["GSPC"]["X_test"]
y_gspc_train = result["GSPC"]["y_train"]
y_gspc_test  = result["GSPC"]["y_test"]

# Assign DJI values
X_dji_train = result["DJI"]["X_train"]
X_dji_test  = result["DJI"]["X_test"]
y_dji_train = result["DJI"]["y_train"]
y_dji_test  = result["DJI"]["y_test"]

# Assign NYSE values
X_nyse_train = result["NYSE"]["X_train"]
X_nyse_test  = result["NYSE"]["X_test"]
y_nyse_train = result["NYSE"]["y_train"]
y_nyse_test  = result["NYSE"]["y_test"]


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
<ipython-input-5-6aa10446cf8a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)
<ipython-input-5-6aa10446cf8a>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Date'] = pd.to_datetime(df['Date'])
<ipython-input-5-6aa10446cf8a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the cave

In [ ]:
print(X_dji_train)

Price              Close
Ticker              ^DJI
Date                    
1994-01-03   3756.600098
1994-01-04   3783.899902
1994-01-05   3798.820068
1994-01-06   3803.879883
1994-01-07   3820.770020
...                  ...
2012-07-10  12653.120117
2012-07-11  12604.530273
2012-07-12  12573.269531
2012-07-13  12777.089844
2012-07-16  12727.209961

[4669 rows x 1 columns]


In [ ]:
print(X_dji_train)

Price              Close
Ticker              ^DJI
Date                    
1994-01-03   3756.600098
1994-01-04   3783.899902
1994-01-05   3798.820068
1994-01-06   3803.879883
1994-01-07   3820.770020
...                  ...
2012-07-10  12653.120117
2012-07-11  12604.530273
2012-07-12  12573.269531
2012-07-13  12777.089844
2012-07-16  12727.209961

[4669 rows x 1 columns]


In [58]:
from statsmodels.tsa.stattools import adfuller

# Function to calculate log returns
def calculate_log_returns(prices):
    return np.log(prices + 1e-8 / prices.shift(1) +1e-8).fillna(0)


def garch_predictions(prices_series, forecast_horizon=1):
    mean_predictions = []
    volatility_predictions = []
    prediction_dates = []
    true_volatility = []

    prices_series = prices_series.sort_index()

    # Calculate log returns and scale them
    log_returns = calculate_log_returns(prices_series) * 100  # Scale log returns

    # Check if log returns are constant
    if np.all(log_returns == log_returns.iloc[0]):
        raise ValueError("Log returns are constant. Check the input data.")

    adf_result = adfuller(log_returns.dropna())
    print(f"ADF Statistic: {adf_result[0]}")
    print(f"p-value: {adf_result[1]}")

    if adf_result[1] > 0.05:
        print("Warning: Log returns may not be stationary.")

    for i in range(90, len(prices_series)):
        window_prices = prices_series.iloc[i-90:i]
        window_log_returns = calculate_log_returns(window_prices) * 100  # Scale log returns

        model = arch_model(window_log_returns, mean='constant', vol='GARCH', p=1, q=1, rescale=False)
        options = {
        'maxiter': 1000,  # Maximum number of iterations
        'ftol': 1e-6,     # Tolerance for convergence
        'disp': False     # Disable verbose output
        }
        model_fit = model.fit(disp='off')

        forecast = model_fit.forecast(horizon=forecast_horizon)

        predicted_mean = forecast.mean.iloc[-1].values[0]
        predicted_volatility = np.sqrt(forecast.variance.iloc[-1].values[0])

        mean_predictions.append(predicted_mean)
        volatility_predictions.append(predicted_volatility)
        prediction_dates.append(prices_series.index[i])

        # Calculate true volatility using the current log return
        current_log_return = np.log(prices_series.iloc[i] / prices_series.iloc[i-1]) * 100  # Scale log return
        true_volatility.append((current_log_return - predicted_mean) ** 2)

    predictions_df = pd.DataFrame({
        'Date': prediction_dates,
        'Predicted_Mean': mean_predictions,
        'Predicted_Volatility': volatility_predictions,
        'True_Volatility': true_volatility
    }).set_index('Date')

    return predictions_df



In [86]:
# LSTM Model Definition
def create_lstm_model():
    model = models.Sequential([
        layers.LSTM(256, return_sequences=True, input_shape=(90, 1)),
        layers.Dropout(0.2),
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(1)
    ])
    return model

# Custom Loss Function
def custom_loss(predicted_volatility, lambda_param=0.5):
    # Convert predicted_volatility to a tensor here
    predicted_volatility = tf.convert_to_tensor(predicted_volatility, dtype=tf.float32)

    def loss(y_true, y_pred):
        # Reshape predicted_volatility to match y_true
        predicted_volatility_reshaped = tf.reshape(predicted_volatility, tf.shape(y_true))

        # Instantiate MeanSquaredError
        mse = tf.keras.losses.MeanSquaredError()

        # Calculate MSE for true variance and GARCH-predicted variance
        mse_true = mse(y_true, y_pred)
        mse_garch = mse(predicted_volatility_reshaped, y_pred)

        # Combine the losses
        return (lambda_param * mse_true) + ((1 - lambda_param) * mse_garch)

    return loss



In [59]:
# Example Usage
prices_series = X_dji_train
predictions_df = garch_predictions(prices_series)

true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparing the data for LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))]).reshape(-1, 90, 1)
y = np.vstack((true_volatility[90:], predicted_volatility[90:]))  # Ground truth and GARCH predictions


ADF Statistic: -2.2485887735694763
p-value: 0.18908523296431418


/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 8. The message is:
Positive directional derivative for linesearch
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 4. The message is:
Inequality constraints incompatible
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/

In [63]:
# Ensure X and y are of type float32
X = X.astype(np.float32)
y = y.astype(np.float32)

<ipython-input-63-7f1ea2187917>:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  X = X.astype(np.float32)


X shape: (4489, 90, 1), X dtype: float32
y shape: (2, 4489), y dtype: float32
[[[678280.94]
  [678934.94]
  [678595.1 ]
  ...
  [675544.3 ]
  [678964.1 ]
  [678062.  ]]

 [[678934.94]
  [678595.1 ]
  [676821.4 ]
  ...
  [678964.1 ]
  [678062.  ]
  [681158.25]]

 [[678595.1 ]
  [676821.4 ]
  [678368.7 ]
  ...
  [678062.  ]
  [681158.25]
  [679109.5 ]]

 ...

 [[887758.1 ]
  [890709.25]
  [886717.4 ]
  ...
  [895350.06]
  [895954.1 ]
  [895332.25]]

 [[890709.25]
  [886717.4 ]
  [886999.44]
  ...
  [895954.1 ]
  [895332.25]
  [896314.94]]

 [[886717.4 ]
  [886999.44]
  [888002.56]
  ...
  [895332.25]
  [896314.94]
  [891349.94]]]


<ipython-input-63-7f1ea2187917>:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y = y.astype(np.float32)


In [95]:
# Custom Loss Function
def custom_loss(lambda_param=0.5):
    # Convert predicted_volatility to a tensor
    predicted_volatility = tf.convert_to_tensor(predicted_volatility, dtype=tf.float32)

    def loss(y_true, y_pred):
        # Slice predicted_volatility to match the batch size
        batch_size = tf.shape(y_true)[0]
        predicted_volatility_batch = predicted_volatility[:batch_size]

        # Calculate MSE for true variance and GARCH-predicted variance
        mse=tf.keras.losses.MeanSquaredError()
        mse_true = tf.keras.losses.MeanSquaredError(y_true, y_pred)
        mse_garch = tf.keras.losses.MeanSquaredError(predicted_volatility_batch, y_pred)

        # Combine the losses
        return (lambda_param * mse_true) + ((1 - lambda_param) * mse_garch)

    return loss

# LSTM Model Definition
def create_lstm_model():
    model = models.Sequential([
        Input(shape=(90, 1)),  # Define input shape here
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(1)  # Output is a single value (predicted variance)
    ])
    return model


true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparing the data for LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))]).reshape(-1, 90, 1)
y = true_volatility[90:]  # Only true_volatility is passed as y

# Ensure X and y are of type float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# Check shapes and types
print(f"X shape: {X.shape}, X dtype: {X.dtype}")
print(f"y shape: {y.shape}, y dtype: {y.dtype}")

# Model Initialization and Compilation
model = create_lstm_model()
model.compile(optimizer='adam', loss=custom_loss( lambda_param=0.5))

# Model Training
model.fit(X[:,1], y[:,1], epochs=100, batch_size=32, shuffle=True)

<ipython-input-95-7aca431b7774>:46: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  X = X.astype(np.float32)


X shape: (4489, 90, 1), X dtype: float32
y shape: (4489,), y dtype: float32


<ipython-input-95-7aca431b7774>:47: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  y = y.astype(np.float32)


UnboundLocalError: cannot access local variable 'predicted_volatility' where it is not associated with a value

In [99]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, Input

# Custom Loss Function (if needed)
def custom_loss(lambda_param=0.5):
    def loss(y_true, y_pred):
        # Your custom loss implementation
        pass
    return loss

# LSTM Model Definition
def create_lstm_model():
    model = models.Sequential([
        Input(shape=(90, 1)),  # Define input shape here
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256, return_sequences=True),
        layers.Dropout(0.2),
        layers.LSTM(256),
        layers.Dense(128),
        layers.BatchNormalization(),
        layers.ReLU(),
        layers.Dense(1)  # Output is a single value (predicted variance)
    ])
    return model


true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparing the data for LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))]).reshape(-1, 90, 1)
y = np.vstack((true_volatility[90:], predicted_volatility[90:])).T  # Shape: [n_samples, 2]

# Standardize X
scaler_X = StandardScaler()
X_scaled = np.array([scaler_X.fit_transform(seq) for seq in X])  # Standardize each sequence

# Standardize y
scaler_y = StandardScaler()
y_scaled = scaler_y.fit_transform(y)  # Standardize y

# Ensure X and y are of type float32
X_scaled = X_scaled.astype(np.float32)
y_scaled = y_scaled.astype(np.float32)

# Check shapes and types
print(f"X shape: {X_scaled.shape}, X dtype: {X_scaled.dtype}")
print(f"y shape: {y_scaled.shape}, y dtype: {y_scaled.dtype}")

# Model Initialization and Compilation
model = create_lstm_model()
model.compile(optimizer='adam', loss="mse")

# Model Training
model.fit(X_scaled, y_scaled[:, 1], epochs=20, batch_size=32)

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  array = numpy.asarray(array, order=order, dtype=dtype)
/usr/local/lib/python3.11/dist-packages/sklearn/utils/_array_api.py:839: FutureWarning: Calling float o

X shape: (4489, 90, 1), X dtype: float32
y shape: (4489, 2), y dtype: float32
Epoch 1/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 8s 24ms/step - loss: 0.5356
Epoch 2/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 1.8355
Epoch 3/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.1752
Epoch 4/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.1203
Epoch 5/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.6882
Epoch 6/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - loss: 0.9350
Epoch 7/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.1422
Epoch 8/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 0.1861
Epoch 9/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.3055
Epoch 10/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 6s 24ms/step - loss: 2.1350
Epoch 11/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - loss: 0.4611
Epoch 12/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.2347
Epoch 13/20
141/141 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 1.0465
Epoch 14/20
141/141 ━━━━━━━━━━━━━━

In [ ]:
true_volatility = predictions_df['True_Volatility'].values
predicted_volatility = predictions_df['Predicted_Volatility'].values

# Preparing the data for LSTM
X = np.array([true_volatility[i-90:i] for i in range(90, len(true_volatility))]).reshape(-1, 90, 1)
y = np.vstack((true_volatility[90:], predicted_volatility[90:])).T

# Ensure X and y are of type float32
X = X.astype(np.float32)
y = y.astype(np.float32)

# Check shapes and types
print(f"X shape: {X.shape}, X dtype: {X.dtype}")
print(f"y shape: {y.shape}, y dtype: {y.dtype}")

# Model Initialization and Compilation
model = create_lstm_model()
model.compile(optimizer='adam', loss="mse")

# Model Training
model.fit(X[:,1], y[:,1], epochs=100, batch_size=32, shuffle=True)

In [ ]:
# Step 1: Predict standardized volatilities using the LSTM model
y_pred_scaled = model.predict(X_scaled)  # Shape: (n_samples, 1)

# Step 2: Inverse transform the predicted volatilities to the original scale
# Create a dummy array with the same shape as y_scaled
dummy_array = np.zeros_like(y_scaled)
dummy_array[:, 1] = y_pred_scaled.flatten()  # Replace the second column with predicted volatilities

# Inverse transform the dummy array
y_pred_inverse = scaler_y.inverse_transform(dummy_array)  # Shape: (n_samples, 2)

# Extract the inverse-transformed predicted volatilities
predicted_volatility_original_scale = y_pred_inverse[:, 1]  # Shape: (n_samples,)

# Step 3: Combine predicted volatilities with corresponding means
# Assuming you have the means stored in a variable (e.g., `means`)
means = predictions_df['Predicted_Mean'].values[90:]  # Shape: (n_samples,)

# Calculate log returns
log_returns = means + predicted_volatility_original_scale

# Step 4: Store or use the log returns
# For example, you can store them in a DataFrame
output_df = pd.DataFrame({
    'Date': predictions_df.index[90:],  # Use the corresponding dates
    'Predicted_Mean': means,
    'Predicted_Volatility': predicted_volatility_original_scale,
    'Log_Return': log_returns
})

# Display the output DataFrame
print(output_df.head())

In [100]:
# Step 1: Predict standardized volatilities using the LSTM model
y_pred_scaled = model.predict(X_scaled)  # Shape: (n_samples, 1)

# Step 2: Inverse transform the predicted volatilities to the original scale
# Create a dummy array with the same shape as y_scaled
dummy_array = np.zeros_like(y_scaled)
dummy_array[:, 1] = y_pred_scaled.flatten()  # Replace the second column with predicted volatilities

# Inverse transform the dummy array
y_pred_inverse = scaler_y.inverse_transform(dummy_array)  # Shape: (n_samples, 2)

# Extract the inverse-transformed predicted volatilities
predicted_volatility_original_scale = y_pred_inverse[:, 1]  # Shape: (n_samples,)

# Step 3: Combine predicted volatilities with corresponding means
# Assuming you have the means stored in a variable (e.g., `means`)
means = predictions_df['Predicted_Mean'].values[90:]  # Shape: (n_samples,)

# Calculate log returns
log_returns_pred = means + predicted_volatility_original_scale

# Step 4: Store or use the log returns
# For example, you can store them in a DataFrame
output_df = pd.DataFrame({
    'Date': predictions_df.index[90:],  # Use the corresponding dates
    'Predicted_Mean': means,
    'Predicted_Volatility': predicted_volatility_original_scale,
    'Log_Return': log_returns_pred
})

# Display the output DataFrame
print(output_df.head())

141/141 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step
        Date  Predicted_Mean  Predicted_Volatility     Log_Return
0 1994-09-20      823.590281          523728.15625  524551.746531
1 1994-09-21      823.627933          523737.25000  524560.877933
2 1994-09-22      823.659410          523760.09375  524583.753160
3 1994-09-23      823.685647          523973.06250  524796.748147
4 1994-09-26      823.094873          523740.43750  524563.532373
